<center><img src="../../img/ods_stickers.jpg"/> Обнаружение мошенничества с кредитными картами</center>
<center>Индивидуальный проект. Автор: Александр Евгеньевич Ширкин. Slack: @panchos39</center>
<center>https://www.kaggle.com/mlg-ulb/creditcardfraud</center>

## Обнаружение мошенничества с кредитными картами
Индивидуальный проект. Автор: Александр Евгеньевич Ширкин. Slack: @panchos39
https://www.kaggle.com/mlg-ulb/creditcardfraud

### 1. Описание набора данных и признаков

В данном проекте решается задача поиска мошенничества в множестве транзакций с кредитных карт. В наше время, когда большинство деловых сделок совершается электронным способом, все банки заинтересованы в безопасности счетов своих клиентов, и быстрая реакция на мошеннические сделки позволит улучшить качество услуг предоставляемых банком и престижность банка для вкладчиков

Датасет содерижит транзакции сделанные европейцами с кредитных карт с сентября 2013-го года.  В датасете присутствуют такие транзакции, которые проходили в течении двух дней, и из них 492 мошеннические и 284,807 легальные. Данный набор данных очень несбалансирован, целевой класс (мошенничество) составляет всего лишь 0.172% от доли всех транзакций.

В данных содержатся только вещественные признаки, которые являются результатом PCA преобразования. К сожалению, из за проблем с конфедициальностью, в данных нет оригинальных признаков и другой косвенной информации о данных. Единственные признаки, которые не были преобразованы, это 'Time' и 'Amount'.

<b>Признаки V1, V2, ..., V28 - главные компоненты, полученные после преобразования PCA исходных признаков.</b> 

 

<b>Признак 'Time' содержит время в секундах между каждой транзакцией и самой первой транзакцией в наборе данных.</b>

<b>Признак 'Amount' - это объем транзакции. Данный признак может использоваться для обучения с учетом издержек классификации.</b>

<b>Признак 'Class' - целевая переменная, она принимает значение 1 - если транзакция имеет мошеннический характер, и 0 - если транзакция легальная</b>


### 2. Первичный анализ данных

#### Подключаем библиотеки

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # to plot graph
import seaborn as sns
import time
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, validation_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score, recall_score, f1_score, precision_recall_curve
from sklearn.linear_model import LogisticRegression
sns.set_style("darkgrid")
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from scipy.sparse import hstack, vstack
import warnings
warnings.filterwarnings('ignore')

#### Считываем наш набор данных

In [ ]:
PATH_TO_DATA = '../../data/creditcard/'
RANDOM_STATE = 17

In [ ]:
data = pd.read_csv(os.path.join(PATH_TO_DATA, 'creditcard.csv.zip'))

#### Теперь исследуем набор данных

In [ ]:
data.info()

1. Мы можем увидеть, что структурно данные состоят из 284,807 строк и 31 столбца.
2. Все признаки вещественные, кроме целевой целочисленной переменной
3. В данных нет пропусков.
4. Время также представленно в виде вещественного числа и представляет время в секундах с начала первой транзакции

Признак 'Time' не так хорошо интерпретируется, когда время дано в секундах. Давайте предположим, что транзакции начинаются с 00:00 и заканчиваются в 23:59. Таким образом переведем секунды в часы дня

In [ ]:
data['hour'] = data['Time'].apply(lambda x: np.ceil(float(x)/3600) % 24)

Т.к. данные собраны с европейского банка, то будет логичным судить, что часовые пояса в Европейских странах не сильно смещены относительно друг друга, и должно оправдаться предположение, что с 1 часа ночи до 5 утра должно наблюдаться сокращение числа транзакций(ночное время) и примерно с 6 часа снова начинается повышение числа транзакций. Проверем наше предположение при помощи сводной таблицы

In [ ]:
data.pivot_table(values='Amount',index='Class',columns='hour',aggfunc='count')

Как мы можем увидеть, наше предположение оправдалось. С достаточно большой долей уверенности мы можем судить о том, что данные действительно начинаются с 00:00, и мы можем оправданно создать признак, время в часах. В следующем разделе мы визуально подтвердим наше предположение

In [ ]:
data.describe()

In [ ]:
data['Class'].value_counts()

- Мы видим явный  сильный дисбаланс в распределении целевой переменной

- Посмотрим на распределение целевой переменной в процентном соотношении

In [ ]:
def transaction_dist(labels) :
    # now let us check in the number of Percentage
    Count_Normal_transacation = len(labels[labels ==0 ])
    Count_Fraud_transacation = len(labels[labels == 1])
    Percentage_of_Normal_transacation = Count_Normal_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
    print("Процентное соотношение легальных транзакций",Percentage_of_Normal_transacation*100)
    Percentage_of_Fraud_transacation= Count_Fraud_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
    print("Процентное соотношнеие мошеннических транзакций",Percentage_of_Fraud_transacation*100)

In [ ]:
transaction_dist(data["Class"])

- Теперь мы убедились, что, действтительно, доля мошеннических транзакций составляет всего лишь 0.172% от всех транзакций, что очень мало. 
- Это говорит о сильном дисбалансе в распределении целевой переменной
- Для качественного обучения необходимо использовать методы балансирования выборки , либо взвешивание в модели

Посмотрим как признаки коррелируют друг с другом. Построим матрицу корреляций Пока исключим целевую переменную

In [ ]:
data.drop('Class', axis=1).corr()

Признаков достаточно много, и тяжело судить о взаимодействии признаков без визуализации. Но в данный момент мы можем увидеть следующее, 
- признаки V1, V2, ..., V28 абсолютно не коррелируют друг с другом
- Это частично подтверждает тот факт, что признаки являются главными компонентами PCA преобразования, т.к. главные компоненты PCA преобразования являются линейно независимыми векторами, и каждый вектор описывает свою долю дисперсии в данных

Посмотрим как признаки коррелируют с целевой переменной. Исключим признак 'Time', т.к. мы его рассмотрим отдельно

In [ ]:
data.drop(['Class'], axis=1).corrwith(data['Class']).sort_values(axis=0)

Можем увидеть , что есть признаки, которые достаточно сильно трицательно коррелируют, и признаки, которые положительно коррелируют с целевой переменной, но не так сильно. Будем считать корреляцию значимой, если |corr| > 0.1

Среди отрицательных корреляций можем выделить признаки( выберем порог, где корреляция меньше 0.1)
- V17, V14, V12, V10, V16, V3, V7, V18, V1

Cреди положительных корреляций (выберем порог, где корреляция больше 0.1)
- V4, V11

Разумеется это наши первые предположения о влиянии признаков на целевую переменную и мы пока не делаем общих выводов. 

Возможно окажется так, что есть комбинации признаков, которые будут коррелировать намного сильнее, чем текущие показатели

### 3. Первичный визуальный анализ данных



Построены визуализации (распределения признаков, матрица корреляций и т.д.), описана связь с анализом данным (п. 2). Присутствуют выводы;

Посмотрим на распределение целевой переменной

In [ ]:
plt.figure(figsize=(5, 10))
sns.countplot("Class",data=data)
plt.show()

- Теперь мы можем визуально подтвердить, что в данных наблюдается сильный дисбаланс распределения целевой переменной
- Процентное соотношение легальных транзакций 99.82725143693798 %
- Процентное соотношнеие мошеннических транзакций 0.1727485630620034 %
- Для обучения необходима балансировка выборки

Посмотрим, как распределн объем транзакция на легальные и мошеннические транзакции

In [ ]:
Fraud_transacation = data[data["Class"]==1]
Normal_transacation= data[data["Class"]==0]
plt.figure(figsize=(10,6))
plt.subplot(1, 2, 1)
Fraud_transacation.Amount.plot.hist(title="Мошеннические транзакции")
plt.subplot(1,2,2)
Normal_transacation.Amount.plot.hist(title="Легальные транзакции")
plt.show()

- Распределение легальных транзакций трудно интерпретировать. 
- Судя по гистограмме, все легальные транзакции проходят объемом меньше 2.5 тысяч у.е
- Уточним данные для гистограммы, ограничив объем транзакции 2.5 к. у.е

In [ ]:
Fraud_transacation = data[data["Class"]==1]
Normal_transacation= data[data["Class"]==0]
plt.figure(figsize=(10,6))
plt.subplot(1, 2, 1)
Fraud_transacation[Fraud_transacation["Amount"]<= 2500].Amount.plot.hist(title="Мошеннические транзакции")
plt.subplot(1, 2, 2)
Normal_transacation[Normal_transacation["Amount"]<=2500].Amount.plot.hist(title="Легальные транзакции")
plt.show()

Вывод :
- по объему транзакций на долю легальных и мошеннических транзакций мы не видим каких то явных закономерностей
- По объему транзакций нельзя строго судить о характере проведенной операции

<b>Посмотрим, как распределены легальные и мошеннические транзакции по часам дня</b>

In [ ]:
def PlotHistogramHour(df,norm):
    bins = np.arange(df['hour'].min(),df['hour'].max()+2)
    plt.figure(figsize=(15,4))
    sns.distplot(df[df['Class']==0.0]['hour'],
                 norm_hist=norm,
                 bins=bins,
                 kde=False,
                 color='b',
                 hist_kws={'alpha':.5},
                 label='Легальные')
    sns.distplot(df[df['Class']==1.0]['hour'],
                 norm_hist=norm,
                 bins=bins,
                 kde=False,
                 color='r',
                 label='Мошеннические',
                 hist_kws={'alpha':.5})
    plt.xticks(range(0,24))
    plt.legend()
    plt.show()

In [ ]:
print('Частотная гистограма Легальных/Мошеннических транзакций на каждый час дня')
PlotHistogramHour(data,False)
print('*На гистограмме не видно мошеннических транзакций, т.к. их очень мало, необходимо нормализовать гистограммы.\n')
print('Нормализованная гистограма Легальных/Мошеннических транзакций на каждый час дня')

PlotHistogramHour(data,True)

Вывод: 
- время, когда была совершена транзакция, несомненно вносит определенный вклад в целевую переменную, и есть предпоссылки для того, чтобы ее использовать в модели. 
- Возможно это будет не сам час, а какой то конкретный интервал суток, например, ночь, что видно по нормализованной гистограмме

In [ ]:
sns.pairplot(data=pd.concat([data.loc[:,'hour'],data.loc[:,'V1':'V6'],data.loc[:,'Class']],axis=1),
             hue='Class',
             diag_kind='kde',
             plot_kws={'alpha':0.2})
plt.show()

### 4. Инсайты, найденные зависимости

- Мы выяснили, время в секундах начинается с полуночи (с 00:00), значит можно справедливо определить время в часах, когда была совершена транзакция
- Мы выяснили, что наблюдается повышенная мошенническая активность в ночное время суток

### 5. Выбор метрики

Для того что определить метрику качества для нашей задачи нужно обратить внимание на следующие факторы:
1. Какая решается задача
2. Сколько классов
3. Присутствует ли дисбаланс в распределении целевого признака
4. Какова главная цель решения задачи

Так как мы решаем задачу бинарной классификации, то имеет смысл рассматривать следующие метрики:
    
1. Accuracy = TP+TN/Total - доля верных ответов
2. Precison = TP/(TP+FP) - Точность
3. Recall = TP/(TP+FN) - Полнота
4. AUROC - Area Under ROC - Площадь под ROC кривой
5. F-score - F-мера (Взвешенное среднее точности и полноты)


- TP = True possitive - Истинные срабатываня
- TN = True negative - Истинные пропуски
- FP = False possitve - Ложные срабатывания
- FN= False Negative - Ложные пропуски

1. Так как мы наблюдаем явный дисбаланс в распределении целевой переменной, то доля верных ответов не подходит для оценки качества модели. Мы можем получить достаточно большую долю верных ответов, если просто скажем, что все транзакции легальные
2. Точность - более подходящая метрика оценки, но в данной задаче стоит обратить внимание на то, что нам более важно, точно определять мошеннические транзакции, при этом не охватывая их все полностью, либо же мы охватываем все мошеннические транзакции, но некоторые транзакции ошибочно считаем мошенническими, хотя они являются легальными
3. Полнота - для интересов банков, наиболее важный параметр в данной задаче является полнота, т.к. не будет серьезных последствий, если мы предупредим клиента о возможности мошеннических операций на его счете, даже если их на самом деле нет, клиент все равно будет доволен, что банк следит за его активами, в обратном же случае, мы упускаем много потенциальных мошеннических транзакций, и клиент может остаться не доволен услугами банка. Поэтому полнота является адекватной метрикой качества модели, но есть еще один подходящий вариант.
4. Площадь под ROC кривой - хорошая метрика, если мы стремимся равноценно хорошо увеличивать как Точность, так и Полноту, но в нашем случае, это может привести к очень хорошей точности, но при этом Полнота будет немного меньше, хотя в нашем случае, намного лучше будет, если Полнота будет больше.
5. F-мера - можно сказать идеальная метрика для данной задачи, т.к. мы можем указать приоритет, в чем мы больше нуждаемся, в Точности или Полноте, и получить результат, который будет учитывать как Точность, так и Полноту, но при этом больший приоритет будет отдаваться Полноте.

Из наших рассуждений следует, что самые подходящие метрики качества для данной задачи:
- Полнота
- F-мера c приоритетом к Полноте

### 6. Выбор модели




Мы решаем задачу классификации, соответственно в качестве модели могут подойти следующие:

1. Логистическая регрессия
2. Случайный лес
3. Градиентный бустинг
4. Нейронные сети

В наше задаче нет особых ограничений на применение различных моделей.
* Признаков не много, т.е. методы основанные на деревьях решений подходят
* Признаки масштабированы, и не коррелируют друг с другом, также можно применять логистическую регрессию
* Нейронные сети также можно применить, но лично по моему мнению, данный тип моделей будет слишком избыточен, т.к. можно построить достаточно адекватную модель с хорошим отбором признаков, следуя принципу Бритвы Окама. Чем проще, тем лучше. В добавок мы получим эффективность на стадии production, если обратим интерес к более элементарным алгоритмам


### 9. Создание новых признаков и описание этого процесса 

Мы создали новый признак - час дня

In [ ]:
data['hour'] = data['Time'].apply(lambda x: np.ceil(float(x)/3600) % 24)

Описание предпосыок к этому признаку было рассмотрено выше

### 7. Предобработка данных


Мы проделаем следующие действия с нашими данными:
- Отмасштабируем признак Amount для логистической регрессии
- Применеим One Hot Encoding преобразование к признаку hour
- Также рассмотрим два подхода работы с несбалансированными выборками: OverSampling и UnderSampling, и посмотрим, что работает лучше
- Признак Time уберем из выборки, т.к. он не несет статистической значимости, мы определили более полезный признак, время в часах, что лучше описывает данные
- Остальные признаки оставим без изменений, так как они и так являются результатом PCA преобразования

In [ ]:
X, y = data.drop(['Class', 'Time'], axis=1), data['Class']

In [ ]:
X.head()

In [ ]:
train_size = 0.7
n = X.shape[0]

Разделим наши данные на обучающую и тестовую выборку, причем сохраним порядок появляения данных, т.е. в тестовой выборке у нас присутствуют только более поздние данные, чтобы наша модель могла определять мошеннические транзакции на разных временных промежутках

In [ ]:
X_train, y_train = X.iloc[:int(n * train_size)], y.iloc[:int(n * train_size)]
X_test, y_test = X.iloc[int(n * train_size):], y.iloc[int(n * train_size):]

In [ ]:
X_train.head()

In [ ]:
transaction_dist(y_train)

In [ ]:
transaction_dist(y_test)

In [ ]:
ss = StandardScaler()
X_train.Amount = pd.Series(ss.fit_transform(X_train.Amount.values.reshape(-1, 1)).flatten(), name="Amount", index=X_train.index)
X_test.Amount = pd.Series(ss.transform(X_test.Amount.values.reshape(-1, 1)).flatten(), name="Amount", index=X_test.index)

In [ ]:
X_test.head()

Применим One Hot Encoding для признака hour

In [ ]:
ohe = OneHotEncoder(categorical_features='all')
time_ohe_train = ohe.fit_transform(X_train['hour'].values.reshape(-1, 1))
time_ohe_test = ohe.transform(X_test['hour'].values.reshape(-1, 1))

In [ ]:
X_train_csr = hstack((X_train.drop('hour', axis=1).values, time_ohe_train))
X_test_csr = hstack((X_test.drop('hour', axis=1).values, time_ohe_test))

In [ ]:
X_test_csr.shape

#### Реализуем два метода борьбы с несбалансированностью в распределении целевой переменной
#### UnderSampling и OverSampling
#### И протестируем какой из методов будет лучше работать

In [ ]:
rus = RandomUnderSampler(random_state=17, return_indices=True)
ros = RandomOverSampler(random_state=17,  ratio='minority')

In [ ]:
X_train_undersampled, y_train_undersampled, idx_train = rus.fit_sample(X_train_csr, y_train)
X_train_oversampled, y_train_oversampled = ros.fit_sample(X_train_csr, y_train)

In [ ]:
type(X_train_undersampled)

In [ ]:
print ("UnderSampling")
transaction_dist(y_train_undersampled)
print ("--------------------------------------------------------")
print ("OverSampling")
transaction_dist(y_train_oversampled)

### 8. Кросс-валидация и настройка гиперпараметров модели

- Проведем кросс валидацию для логистической регрессии как для UnderSampling так и OverSampling
- будем перебирать различные значения параметра регуляризации C
- В качестве метода разбиения, возьмем стратифицированный, т.к. нам необходимо сохранить баланс классов в каждом фолде
- в случае с OverSampling поставим количество фолдов равное 5-ти, а в UnderSampling - 3-м, т.к. данныых очень мало, и большое количество фолдов может повлиять на качество обучения
- Для воспроизводимости решения, зафиксируем seed равный 17-ти

In [ ]:
lr = LogisticRegression(random_state=17, n_jobs=-1)

In [ ]:
skf = StratifiedKFold(n_splits=3, random_state=17, shuffle=True)

In [ ]:
grid_params = {'C' : np.linspace(start=0.0001, stop=10, num=50)}

In [ ]:
grid_under = GridSearchCV(estimator=lr, param_grid=grid_params, cv=skf, n_jobs=-1, scoring='recall', verbose=10)
grid_over = GridSearchCV(estimator=lr, param_grid=grid_params, cv=skf, n_jobs=-1, scoring='recall', verbose=10)

In [ ]:
grid_under.fit(X_train_undersampled, y_train_undersampled)

In [ ]:
grid_over.fit(X_train_oversampled, y_train_oversampled)

In [ ]:
grid_under.best_score_, grid_under.best_params_

In [ ]:
grid_over.best_score_, grid_under.best_params_

Можем увидеть, что намного лучше результат дает undersampling.
Здесь есть большой плюс. Размер нашего датасета существенно сокращается, и обучение проходит очень быстро, что дает большой простор для дальнейших экспериментов, и можно перебирать множество параметров

### 10 Построение кривых валидации и обучения

In [ ]:
param_range = np.linspace(start=0.0001, stop=10, num=50)
train_scores, test_scores = validation_curve(
    lr, X_train_undersampled, y_train_undersampled, param_name='C', param_range=param_range,
    cv=skf, scoring="recall", n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with Logistic Regression")
plt.xlabel("$\gamma$")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

Судя по кривой валидации и обучения можем увидеть слудующее:
- кривая обучения и валидации находятся не далеко друг от друга и ошибка невелика, значит мы можем судить, что наша модель не переобучилась, и не недообучилась
- кривые не сошлись друг к другу, значит добавление новых данных может улучшить модель

### 11. Прогноз для тестовой или отложенной выборке

In [ ]:
def plot_precision_recall_curve(precision, recall) :
    plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.show()

In [ ]:
under_probs = grid_under.predict_proba(X_test_csr)[:, 1]
over_probs = grid_over.predict_proba(X_test_csr)[:, 1]

under_preds = grid_under.predict(X_test_csr)
over_preds = grid_over.predict(X_test_csr)

In [ ]:
precision_under, recall_under, _ = precision_recall_curve(y_true=y_test, probas_pred=under_probs, pos_label=1)
precision_over, recall_over, _ = precision_recall_curve(y_true=y_test, probas_pred=over_probs, pos_label=1)

In [ ]:
plot_precision_recall_curve(precision_under, recall_under)

In [ ]:
plot_precision_recall_curve(precision_over, recall_over)

In [ ]:
print ("Under sampling ROC AUC Score")
print (roc_auc_score(y_true=y_test, y_score=under_probs))
print ("Over sampling ROC AUC Score")
print (roc_auc_score(y_true=y_test, y_score=over_probs))

In [ ]:
print ("Under sampling Recall Score")
print (recall_score(y_true=y_test, y_pred=under_preds))
print ("Over sampling Recall Score")
print (recall_score(y_true=y_test, y_pred=over_preds))

Результаты сравнимы с результатами на кросс - валидации, вдобавок, мы можем увидеть что модель имеет хорошую обобщающую способность, т.к. дает хороший результат

### 12. Выводы

- Как можно увидеть, мы добились качества модели по метрике recall в 100%
по ROC AUC score - 0.979

- Это очень хороший результат, так как мы находим все мошеннические транзакции на тестовой выборке, и немного ошибаемся, определяя легальные транзакции как мошеннические.

- Но с точки зрения бизнеса и банковской сферы это не так страшно

- Стоит отметить, что до добавления признака hour, модель работала существенно хуже, что подтверждает гипотезу, о том, что данный признак вносит большой вклад в целевую переменную, и большинство нелегальных транзакций проходят ночью

Прошу обратить внимание, что мы использовали только модель логистической регрессии, и добились хорошего результата. В последующем улучшении нашего решения, можно попробовать использовать другие методы, такие как случайный лес и градиентный бустинг, что возможно даст нам прирост в precision, и оставит тот же recall